In [ ]:
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!python3 setup.py install


In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Done.


Done.


Done.


Done.


In [40]:
# This is the text that will be spoken.
text = """A First Principles problem can be realized when an existing approach to address an issue can be found.  In short, there exists a known methodology to solve the problem. Methodical disassembling of a computer to 
repair its defective parts may be considered as a first principles problem. """

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

%ls tortoise/voices

IPython.display.Audio('tortoise/voices/tom/1.wav')


In [7]:
%ls tortoise/voices

angie/                freeman/  myself/        tom/            train_grace/
applejack/            geralt/   pat/           train_atkins/   train_kennard/
cond_latent_example/  halle/    pat2/          train_daws/     train_lescault/
daniel/               jlaw/     rainbow/       train_dotrice/  train_mouse/
deniro/               lj/       snakes/        train_dreams/   weaver/
emma/                 mol/      tim_reynolds/  train_empire/   william/


In [32]:
IPython.display.Audio('tortoise/voices/deniro/1.wav')

In [ ]:
# Pick one of the voices from the output above
voice = 'william'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [04:21<00:00, 16.37s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:17<00:00,  1.10s/it]


Transforming autoregressive outputs into audio..


 90%|████████▉ | 358/400 [05:03<00:35,  1.18it/s]

In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..